In [ ]:
import pandas as pd
import re
import string
import pymorphy3
import nltk

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shark\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


______________________________________________________________________________________________________________________________________________________________________________________

Подгатовка данных

In [ ]:
data = pd.read_csv('date/annotated_data.csv')


data.drop(columns=['categories'], inplace=True)

# Не забудьте скачать стоп-слова один раз
nltk.download('stopwords')
morph = pymorphy3.MorphAnalyzer()


# Стоп-слова для русского языка из nltk
stop_words_nltk = set(stopwords.words('russian')) | {'г', 'б', 'огромный', 'большой'}


def clean_text(text):
    punctuation = string.punctuation                                                            # Удаление пунктуации
    
    cleaned_text = text.translate(str.maketrans({key: ' ' for key in punctuation}))             # Применяем преобразование к тексту
    cleaned_text = re.sub(r"[^а-яА-ЯёЁ]", ' ', cleaned_text)                                    # Удаление латинских букв и цифр
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()                                    # Удаление лишних пробелов
    cleaned_text = cleaned_text.lower().strip()                                                 # Приведение к нижнему регистру
    tokens = cleaned_text.split()                                                               # Токенизация (разделение на слова)
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]                 # Лемматизация с помощью pymorphy3
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words_nltk]    # Удаление стоп-слов (используя nltk)
    
    return ' '.join(filtered_tokens)


# Применяем очистку текста к колонке 'comment'
data['comment'] = data['comment'].apply(clean_text)

# Удаляем дубли и пустые строки
data = data[~data.duplicated('comment')]
data = data.dropna(subset=['comment'])
data = data[data['comment'] != '']


data.to_csv('date/processed_data.csv', index=False)